In [1]:
import numpy as np
from math import factorial as fact
from functools import lru_cache
from itertools import product

from utils import * #Contains relevant functions for A(d) such as norm and cardinality
from tqdm.notebook import tqdm

#### First define a function which can compute the coefficient of a cycle $[\gamma_\ell]$ in any product $a\cdot b$ in $A(d)$

In [2]:
@lru_cache(maxsize=1000000) # use memoization since small values will be called repeatedly.
def Lambda(l: int,a: tuple,b: tuple):
    '''
    Computes normalized coefficient of l-cycle in the product a*b.
    '''
    a_arr, b_arr = np.array(a),np.array(b) # convert to arrays in order to apply standard functions
    
    # base case
    if card(a_arr)==0 or card(b_arr)==0 or l<2:
        return fact(l-1)
    
    # recursion formula
    if norm(a_arr)+norm(b_arr)==l-1:
        if magn(a_arr)<l and magn(b_arr)<=l:
            answer = 0
            for i in range(2,min(l+1,len(b)+2)):
                if b[i-2] > 0:
                    answer += (i-1)*partialterm(l,i,b_arr)*Lambda(l-1,a,tuple(partial(i,b)))
            return l*answer // (l-magn(a_arr))

        elif magn(a_arr)<=l and magn(b_arr)<l:
            answer = 0
            for i in range(2,min(l+1,len(a)+2)):
                if a[i-2] > 0:
                    answer += (i-1)*partialterm(l,i,a_arr)*Lambda(l-1,b,tuple(partial(i,a)))
            return l*answer//(l-magn(b_arr))
    return 0

# non-normalized coefficient of l-cycle in the product a*b
cycle_coeff = lambda l,a,b: Lambda(l,a,b)//fact(l-1)

In [3]:
## lambda(9,[2,..],[1,0,0,0,1,...]) = 54
cycle_coeff(9,
            (2,0,0,0,0,0),
            (1,0,0,0,1,0))

54

#### Now extend coefficients from $\ell$-cycles to general cycle types. 

In [4]:
def gen_cdecomp(a: tuple,c: tuple):
    '''
    Generates all c-decompositions of a as a list of pairs (A,A').
    '''
    answer = []
    a_arr = np.array(a)
    nu = np.nonzero(c)[0][0] +2 #smallest non-zero index
    
    # Create a generator for possible values of A:
    intervals = [range(0,min(a[i],nu)*(i<nu)+1) for i in range(0,len(a))]
    cart_prod = product(*intervals)
    
    
    for A in cart_prod:
        A_arr = np.array(A)
        if magn(A_arr)<=nu:
            answer.append((A,tuple(a_arr-A_arr)))
        
    return answer

In [5]:
gen_cdecomp((0,1,2,3),(0,0,2,1))

[((0, 0, 0, 0), (0, 1, 2, 3)),
 ((0, 0, 1, 0), (0, 1, 1, 3)),
 ((0, 1, 0, 0), (0, 0, 2, 3))]

Now define a general procedure for computing coefficients

In [6]:
@lru_cache(maxsize=1000000)
def Lambda2(c,a,b):
    '''
    Computes coefficient of $[c]$ in product $[a]\cdot [b]$
    '''
    if norm(a) + norm(b) != norm(c):
        return 0
    
    if sum(a)==0 and sum(b)==0 and sum(c) == 0:
        return 1

    nu = np.nonzero(c)[0][0] +2
    a_decomps = gen_cdecomp(a,c)
    b_decomps = gen_cdecomp(b,c)
    
    answer = 0
    for A1,A2 in a_decomps:
        for B1,B2 in b_decomps:
            if norm(np.array(A1))+norm(np.array(B1)) == nu-1:
                answer += cycle_coeff(nu,A1,B1) * Lambda2(tuple(c - np.eye(1,len(c),nu-2,dtype=int)[0]), A2,B2)
    return answer

In [7]:
### lambda([1,1,1], [3,0,...], [3,0,...]) = 12
Lambda2((1,1,1),(3,),(3,))

12

## Generate tables of monomials

To compute the product $a\cdot b$ of basis elements we cycle through all cycletypes of relevant norm and compute the coefficient.

In [8]:
def multiply_basis(a: tuple,b: tuple):
    '''
    Given two basis elements a and b, compute the product a*b as a dictionary.
    '''
    product = {}
    
    for cycle_type in generate_cycletypes(norm(a)+norm(b)):
        if coeff := Lambda2(cycle_type,a,b):
            product[cycle_type] = coeff
    
    return product

In [9]:
print("Multiply x*y:", "\n",
      multiply_basis((1,),(2,)))

print("Multiply y*z:", "\n",
      multiply_basis((2,),(3,)))

print("multiply gamma_3 and gamma_4:", "\n",
     multiply_basis((0,1),(0,0,1)))

Multiply x*y: 
 {(3,): 3, (1, 1): 3, (0, 0, 1): 2}
Multiply y*z: 
 {(5,): 10, (3, 1): 9, (2, 0, 1): 6, (1, 2): 9, (1, 0, 0, 1): 5, (0, 1, 1): 6, (0, 0, 0, 0, 1): 3}
multiply gamma_3 and gamma_4: 
 {(0, 1, 1): 1, (0, 0, 0, 0, 1): 6}


#### Once we can multiply basis elements we can extend linearly to general elements

In [10]:
def multiply_generator(generator: tuple, g: dict, d: int=100):
    product = {}
    
    for basis1, coeff1 in g.items():
        for basis2, coeff2 in multiply_basis(generator,basis1).items():
            if magn(basis2) > d:
                pass
            elif basis2 in product.keys():
                product[basis2] += coeff1*coeff2
            else:
                product[basis2] = coeff1*coeff2
    return product

In [11]:
print("Compute xy^2 as y * xy:")
print(multiply_generator((2,), {(3,): 3, (1, 1): 3, (0, 0, 1): 2}))

print("\n")
print("Multiply gamma_3 with gamma_3 and gamma_4:")
print(multiply_generator((0,1), {(0, 1, 1): 1, (0, 0, 0, 0, 1): 6}))

Compute xy^2 as y * xy:
{(5,): 30, (3, 1): 36, (2, 0, 1): 44, (1, 2): 45, (1, 0, 0, 1): 55, (0, 1, 1): 60, (0, 0, 0, 0, 1): 81}


Multiply gamma_3 with gamma_3 and gamma_4:
{(0, 2, 1): 2, (0, 1, 0, 0, 1): 12, (0, 0, 1, 1): 5, (0, 0, 0, 0, 0, 0, 1): 64}


#### Define a procedure for generating all monomials

In [12]:
def generate_monomials(d=12, alphabet="abcdefghijkl", use_cycles=False):
    if use_cycles:
        generators = [(c,(0,)*i + (1,)) for i,c in enumerate(alphabet[0:d])] #cycle generators
    else:
        generators = [(c,(i+1,)) for i,c in enumerate(alphabet[0:d//2])] #(a_2,0,0,...)
    
    monomials = [(i,{j:1}) for i,j in generators]
    uniquenames = set([c for c in alphabet[0:d//2]])
    
    iters = 0
    for name1, generator in tqdm(generators):
        for name2,g in monomials:
            if norm(list(g.keys())[0]) + generator[0] <= d:
                newname = "".join(sorted(name1+name2))
                if newname not in uniquenames:
                    uniquenames.add(newname)
                    prod = multiply_generator(generator, g,d)
                    if prod != {}:
                        monomials.append((newname, prod))
            iters += 1
            if iters>1000:
                break
    return monomials

In [13]:
monomials = generate_monomials(d = 9, use_cycles = True)
print(len(monomials))
monomials[0:12]


67


[('a', {(1,): 1}),
 ('b', {(0, 1): 1}),
 ('c', {(0, 0, 1): 1}),
 ('d', {(0, 0, 0, 1): 1}),
 ('e', {(0, 0, 0, 0, 1): 1}),
 ('f', {(0, 0, 0, 0, 0, 1): 1}),
 ('g', {(0, 0, 0, 0, 0, 0, 1): 1}),
 ('h', {(0, 0, 0, 0, 0, 0, 0, 1): 1}),
 ('i', {(0, 0, 0, 0, 0, 0, 0, 0, 1): 1}),
 ('aa', {(2,): 2, (0, 1): 3}),
 ('ab', {(1, 1): 1, (0, 0, 1): 4}),
 ('ac', {(1, 0, 1): 1, (0, 0, 0, 1): 5})]

#### Format the monomials and sort them by norm

In [14]:
# Turns monomial "aa" into "a^2" etc.
def format_monomname(string):
    counts = {c:0 for c in "abcdefghij"}
    for char in string:
        counts[char] +=1
    s = ""
    for char in "abcdefg":
        if counts[char] == 1:
            s += char
        elif counts[char]>1:
            s += char + "^" + str(counts[char])
    return s


# Computes norm from label in order to sort monomials
def get_norm(monomname):
    answer = 0
    D = {c:(i+1) for i,c in enumerate("abcdefghij")}
    for char in monomname:
        answer += D[char]
    return answer

In [15]:
## Create sorted dictionary of monomials
sorted_monoms = {i:[] for i in range(2,16)}
for name, dic in monomials:
    if len(name)>1:
        n = get_norm(name)
        sorted_monoms[n] += [(format_monomname(name), dic)]

In [16]:
## Get all monomials of norm 5
sorted_monoms[5]

[('ad', {(1, 0, 0, 1): 1, (0, 0, 0, 0, 1): 6}),
 ('a^2c', {(2, 0, 1): 2, (1, 0, 0, 1): 10, (0, 1, 1): 3, (0, 0, 0, 0, 1): 36}),
 ('a^3b',
  {(3, 1): 6,
   (2, 0, 1): 24,
   (1, 2): 18,
   (1, 0, 0, 1): 75,
   (0, 1, 1): 52,
   (0, 0, 0, 0, 1): 216}),
 ('a^5',
  {(3, 1): 180,
   (2, 0, 1): 320,
   (1, 2): 270,
   (1, 0, 0, 1): 625,
   (0, 1, 1): 480,
   (0, 0, 0, 0, 1): 1296}),
 ('bc', {(0, 1, 1): 1, (0, 0, 0, 0, 1): 6}),
 ('ab^2', {(1, 2): 2, (1, 0, 0, 1): 5, (0, 1, 1): 8, (0, 0, 0, 0, 1): 36})]

## Compute relations

In [17]:
from sympy import *

In [18]:
# Define a function which eliminates elements with too large support and finds relations
def relations(d=9, only_number = True,alphabet="abcdefghijkl"):
    generators = alphabet[0:d//2] + "^1234567890"#Generators that vanish in lower dimension d.
    print("Sufficient set of relations in A(" + str(d)+"):", "\n")
    for n in range(2,d):
        key_names = set([])
        monoms = []

        # Get the relevant monomials and basis elements in A(d) of norm n.
        for name, dic in sorted_monoms[n]:
            ## Consider only monomials of _non-zero_ generators:
            monom_non_triv = True
            for char in name:
                if char not in generators:
                    monom_non_triv = False
            if monom_non_triv:
                monoms.append((name,dic))
                # Take all summands with valid support
                for element in dic.keys():
                    if magn(element) <= d: #Sort out elements with support equal to d.
                        key_names.add(element)
        key_names = sorted(list(key_names))
        
        # Create a matrix with coefficients from the monomial equations
        A = np.zeros((len(key_names),len(monoms)), dtype = int)

        for i, key in enumerate(key_names):
            for j, equation in enumerate(monoms):
                if key in (equation[1]).keys():
                    A[i,j] = (equation[1])[key]

        print(r"norm", n, "monomials:" , "$","".join(m[0]+", " for m in monoms)[0:-2], "$")

        null_space = Matrix(A).nullspace()
        #print(np.array(null_space))
        if len(null_space)==0:
            print("\\\\")
        else:
            #print(r"\begin{align*}")
            for row in null_space:
                common_denom = ilcm(*tuple(x.q for x in row))
                relation = ""
                for i,coeff in enumerate(common_denom*row):
                    relation += str(coeff) + ","
                    #if coeff != 0:
                    #    if coeff == 1:
                    #        relation += monoms[i][0] + " + "
                    #    elif coeff <0 and len(relation)>0:
                    #        relation = relation[0:-2] + "-" + " " + str(coeff)[1:] + monoms[i][0] + " + "
                    #    else:
                    #        relation += str(coeff) + monoms[i][0] + " + "
                #print(relation[0:-2] + "&= 0" + "\\\\")
                print("[" + relation[0:-1] + "]" + ",")
            #print("\end{align*}")
    print("\\newpage")

In [19]:
A = relations(d=8)

Sufficient set of relations in A(8): 

norm 2 monomials: $ a^2 $
\\
norm 3 monomials: $ ab, a^3 $
\\
norm 4 monomials: $ ac, a^2b, a^4, b^2 $
\\
norm 5 monomials: $ ad, a^2c, a^3b, a^5, bc, ab^2 $
[-240,92,-21,1,-96,54],
norm 6 monomials: $ a^2d, a^3c, a^4b, a^6, bd, abc, a^2b^2, b^3, c^2 $
[-5472,1232,-87,1,5376,0,0,0,0],
[960,-80,-51,5,0,384,0,0,0],
[0,32,-15,1,0,0,24,0,0],
[-1440,304,-9,-1,0,0,0,96,0],
[5760,-784,-135,17,0,0,0,0,3584],
norm 7 monomials: $ a^3d, a^4c, a^5b, a^7, abd, a^2bc, a^3b^2, b^2c, ab^3, cd, ac^2 $
[-8,1,0,0,0,0,0,0,0,0,0],
[-64,0,1,0,0,0,0,0,0,0,0],
[-512,0,0,1,0,0,0,0,0,0,0],
[-1,0,0,0,8,0,0,0,0,0,0],
[-1,0,0,0,0,1,0,0,0,0,0],
[-8,0,0,0,0,0,1,0,0,0,0],
[-1,0,0,0,0,0,0,8,0,0,0],
[-1,0,0,0,0,0,0,0,1,0,0],
[-1,0,0,0,0,0,0,0,0,64,0],
[-1,0,0,0,0,0,0,0,0,0,8],
\newpage


In [20]:
A = [[-5472,1232,-87,1,5376,0,0,0,0],
     [960,-80,-51,5,0,384,0,0,0],
     [0,32,-15,1,0,0,24,0,0],
     [-1440,304,-9,-1,0,0,0,96,0],
     [5760,-784,-135,17,0,0,0,0,3584],
     [-240,92,-21,1,0,-96,54]
    ]